In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def model_20():    
    learning_rate = 5e-5
    #optimizer = SGD(lr=learning_rate, momentum = 0.9, decay = 1e-3, nesterov = True)
    optimizer = Adam(lr=learning_rate)
    
    inputs = Input(shape=(1, 20, 20, 6))
    
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 1, 1, 1, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 5, 5, 1, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)    
    
    output = Flatten(name='flatten')(conv1)
    output = Dense(150)(output)
    output = PReLU()(output)
    output = BatchNormalization()(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    model3d = Model(inputs, output)
    model3d.compile(loss='binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model3d

def model_30():    
    learning_rate = 5e-5
    #optimizer = SGD(lr=learning_rate, momentum = 0.9, decay = 1e-3, nesterov = True)
    optimizer = Adam(lr=learning_rate)
    
    inputs = Input(shape=(1, 30, 30, 10))
    
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 2, 2, 1, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)    
    
    output = Flatten(name='flatten')(conv1)
    output = Dense(250)(output)
    output = PReLU()(output)
    output = BatchNormalization()(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    model3d = Model(inputs, output)
    model3d.compile(loss='binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model3d

def model_40():    
    learning_rate = 5e-5
    #optimizer = SGD(lr=learning_rate, momentum = 0.9, decay = 1e-3, nesterov = True)
    optimizer = Adam(lr=learning_rate)
    
    inputs = Input(shape=(1, 40, 40, 26))
    
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 2, 2, 2, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(64, 5, 5, 3, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)    
    
    output = Flatten(name='flatten')(conv1)
    output = Dense(250)(output)
    output = PReLU()(output)
    output = BatchNormalization()(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    model3d = Model(inputs, output)
    model3d.compile(loss='binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model3d

### Preprocessing: 
- 1) Data augmentation - translated by 1 voxel along each axis and rotated 90, 180 and 270 degrees with the transverse plane. In total, 0.65 million samples generated for training. 
- 2) Normalization - clipped the intensities into the interval (-1000,400) HU and normalized them to the range of (0,1).

### 3D CNN architecture details: 
- Learning from Scratch, lr=0.3 and decayed by 5% every 5000 iterations. batchsize=200, momentum=0.9, and the dropout rate=0.2 stragety is utilized in C and FC layers.

In [3]:
true_path = PATH['cls_train_true']
false_path = PATH['cls_train_false']
model_paths = PATH['model_paths']


In [6]:
def train_generator(size):
    file_list_true = sorted(glob.glob(true_path + "*_3d_20_6_i.npy"))
    file_list_false = sorted(glob.glob(false_path + "*_3d_20_6_i.npy"))
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,size[0],size[1],size[2]])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):
        cc= np.load(file_list_true[i]).reshape([1,size[0],size[1],size[2]])
        sample[i] = cc[0]
        labels[i][0] = 1
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_true[i]).reshape([1,size[0],size[1],size[2]])
        sample[j+len(file_list_true)] = bb[0]
        labels[j+len(file_list_true)][1] = 1    
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, size, check_name = None):
    
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 15, 
                                   verbose = 1),
    ModelCheckpoint(model_paths + '{}.h5'.format(name), 
                        monitor='val_loss', 
                        verbose = 0, save_best_only = True)]
    
    if check_name is not None:
        check_model = model_paths + '{}.h5'.format(check_name)
        model = load_model(check_model, 
                           custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})
    else:
        model = model_20()

    model.fit_generator(train_generator(size), nb_epoch = 150, verbose = 1, 
                        callbacks = callbacks,
                        samples_per_epoch = 551, nb_val_samples = 50)
    return model

In [7]:
new_model = True

if new_model: 
    learning_rate = 8e-6
    fenlei_fit('final_fenlei', [20,20,6])
else:
    learning_rate = 1e-5
    fenlei_fit('final_fenlei', [20,20,6], 'final_fenlei')

100%|██████████| 44541/44541 [00:08<00:00, 5308.48it/s]


Epoch 1/150


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/mahui/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/mahui/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/mahui/anaconda/lib/python2.7/site-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
TypeError: tuple object is not an iterator



ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None